## Assignment 4A - Vector Addition

In [1]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
code = """

#include <stdio.h>
#include <stdlib.h>

#define N 1000000

__global__ void add(int *a, int *b, int *c) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    if (tid < N) {
        c[tid] = a[tid] + b[tid];
    }
}

int main() {
    int *a, *b, *c;
    int *d_a, *d_b, *d_c;
    int size = N * sizeof(int);

    // Allocate memory on host
    a = (int*)malloc(size);
    b = (int*)malloc(size);
    c = (int*)malloc(size);

    // Initialize arrays
    for (int i = 0; i < N; i++) {
        a[i] = i;
        b[i] = i * 2;
    }

    // Allocate memory on device
    cudaMalloc(&d_a, size);
    cudaMalloc(&d_b, size);
    cudaMalloc(&d_c, size);

    // Copy data from host to device
    cudaMemcpy(d_a, a, size, cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, b, size, cudaMemcpyHostToDevice);

    // Launch kernel with 1 million threads
    add<<<(N + 255) / 256, 256>>>(d_a, d_b, d_c);

    // Copy result from device to host
    cudaMemcpy(c, d_c, size, cudaMemcpyDeviceToHost);

    // Print first and last elements of result
    printf("c[0]=%d, c[%d] = %d",c[0],N-1,c[N-1]);

    // Free memory
    free(a);
    free(b);
    free(c);
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}
"""

In [ ]:
text_file = open("assign4.cu","w")
text_file.write(code)
text_file.close()

In [ ]:
!nvcc assign4.cu

In [ ]:
!./a.out

c[0]=0, c[999999] = 2999997

In [ ]:
!nvprof ./a.out

==703== NVPROF is profiling process 703, command: ./a.out
==703== Profiling application: ./a.out
c[0]=0, c[999999] = 2999997==703== Profiling result:
            Type  Time(%)      Time     Calls       Avg       Min       Max  Name
 GPU activities:   49.67%  1.5313ms         2  765.64us  726.81us  804.47us  [CUDA memcpy HtoD]
                   48.79%  1.5040ms         1  1.5040ms  1.5040ms  1.5040ms  [CUDA memcpy DtoH]
                    1.54%  47.519us         1  47.519us  47.519us  47.519us  add(int*, int*, int*)
      API calls:   94.94%  122.18ms         3  40.726ms  65.240us  122.04ms  cudaMalloc
                    3.70%  4.7670ms         3  1.5890ms  948.72us  2.8387ms  cudaMemcpy
                    0.78%  1.0041ms         1  1.0041ms  1.0041ms  1.0041ms  cuDeviceGetPCIBusId
                    0.43%  555.92us         3  185.31us  145.86us  205.17us  cudaFree
                    0.09%  119.83us       101  1.1860us     137ns  51.243us  cuDeviceGetAttribute
                    